In [1]:
import numpy as np
import xarray as xr
import os
import fnmatch
from tqdm.autonotebook import tqdm
import dask
import random
import sys
import gcsfs
fs = gcsfs.GCSFileSystem() # equivalent to fsspec.fs('gs')
sys.path.insert(0, '/home/jovyan/CMIP6cf/cmip6cf/')

from dependence_metrics import kendallstau, utdc_at_threshold, utdc_cfg

/tmp/ipykernel_1301/3902141566.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def pot(da,threshold,dim):
    
    assert (threshold>=0) & (threshold<1)
    
    return da.where(da>da.quantile(threshold,dim=dim))

def declustered_peaks(da,threshold,window_len,dim):
    #computes peaks above threshold of xr.DataArray and declusters them with a rolling window.
    
    peaks = pot(da,threshold,dim)
    
    return peaks.where(peaks==peaks.rolling({dim:window_len},center=True,min_periods=1).max(skipna=True))

def rolling_max(da,window_len,dim):
    return da.rolling({dim:window_len},center=True,min_periods=1).max()

def compute_ktau_in_windows(da1,da2,dim):

    ktau = xr.apply_ufunc(
                    kendallstau, da1, da2,
                    input_core_dims=[[dim], [dim]], #core dimension: time, loop over the others
                    output_core_dims=[["statistic"]], #outputs tau and p
                    vectorize=True, 
                    dask='allowed', #allow calculating in chunks (dask='parallelized' doesn't work)
                    output_dtypes=[float],
                    output_sizes={"statistic": 2}, #output must be numpy array
                    )
    return ktau

def compute_utdc_in_windows(da1,da2,dim,estimator):

    if estimator=='cfg':
        utdc = xr.apply_ufunc(
                    utdc_cfg, da1, da2,
                    input_core_dims=[[dim], [dim]], #core dimension: time, loop over the others
                    output_core_dims=[["statistic"]], #outputs tau and p
                    vectorize=True, 
                    dask='allowed', #allow calculating in chunks (dask='parallelized' doesn't work)
                    output_dtypes=[float],
                    output_sizes={"statistic": 2}, #output must be numpy array
                    )
    elif estimator=='threshold':
        utdc = xr.apply_ufunc(
                    utdc_at_threshold, da1, da2, #uses .95% by default
                    input_core_dims=[[dim], [dim]], #core dimension: time, loop over the others
                    output_core_dims=[["statistic"]], #outputs tau and p
                    vectorize=True, 
                    dask='allowed', #allow calculating in chunks (dask='parallelized' doesn't work)
                    output_dtypes=[float],
                    output_sizes={"statistic": 2}, #output must be numpy array
                    )
    else:
        raise Except('Estimator'+ str(estimator) +' not implemented.') 
    
    #to-do: implement p-value for utdc!
    
    return utdc

Configure the bivariate sampling:

In [3]:
max_lag = 2 #days
declus_window_len = 1 #days
threshold = .97 #quantile

output_yrs = np.arange(1880,2100,20)
window_len=40 #may need to increase? indicate settings in output folder?
utdc_threshold = .95

season = 'year' #'DJF', 'JJA', 'year'

overwrite_output = True

random.seed(10)

Analyze surge & pr, looping over models & experiments:

In [5]:
surge_dir = '/home/jovyan/CMIP6cf/output/surge_timeseries/'
pr_dir = '/home/jovyan/CMIP6cf/output/pr_timeseries/'

source_ids = list(set(os.listdir(surge_dir)) & set(os.listdir(pr_dir))) #intersection of models

for source_id in [k for k in source_ids if ~k.startswith('.')]: #loop over models
    
    surge_path = os.path.join(surge_dir,source_id)
    pr_path = os.path.join(pr_dir,source_id)
    
    surge_exps = [s.split('_')[-1][0:-3] for s in os.listdir(surge_path) if s.startswith('.')==False]
    pr_exps = [s.split('_')[-1][0:-3] for s in os.listdir(pr_path) if s.startswith('.')==False]
    
    experiment_ids = list(set(surge_exps) & set(pr_exps))

    for experiment_id in experiment_ids: #loop over experiments
        #load data
        fn = fnmatch.filter(os.listdir(surge_path),'*'+experiment_id+'*')[0]
        print('Processing file: '+fn)
        surge_pr = xr.open_mfdataset((os.path.join(surge_path,fn),os.path.join(pr_path,fn)),chunks={'member_id':1,'time':100000,'tg':109})
        
        #generate output paths & check if output already exists
        model_path = os.path.join('/home/jovyan/CMIP6cf/output/surge_pr_dependence/40yr_p97_declus1d',surge_pr.source_id)
        output_fn = os.path.join(model_path,fn)

        if not os.path.exists(model_path):
            os.mkdir(model_path)

        if not overwrite_output: #if not overwriting existing output
            if os.path.exists(output_fn):
                print('Output already exists for this instance.')
                continue
        
        #construct time window indices
        if len(np.unique(surge_pr.time.resample(time='1Y').count()))>1: #remove leap days so that each computation window has the same length
            with dask.config.set(**{'array.slicing.split_large_chunks': True}):
                surge_pr = surge_pr.sel(time=~((surge_pr.time.dt.month == 2) & (surge_pr.time.dt.day == 29))) #^probably (hopefully) only has a small effect on the results
        
        #select DJF:
        days_in_year = int(surge_pr.time.resample(time='1Y').count()[0])
        
        if window_len%2 !=0: #odd
            window_start_idx = days_in_year*(output_yrs-1850-int(np.floor(window_len/2)))
            first_window_idx = np.arange(0*days_in_year,window_len*days_in_year)
        else: #even
            window_start_idx = days_in_year*(output_yrs-1850-int(window_len/2)+1)
            first_window_idx = np.arange(0*days_in_year,window_len*days_in_year)
        
        if np.max(first_window_idx[:,np.newaxis]+window_start_idx[np.newaxis,:])>=len(surge_pr.time):
            raise Exception('Windows exceed simulation length.')
            
        window_idx = xr.DataArray( #indices of windows
            data=first_window_idx[:,np.newaxis]+window_start_idx[np.newaxis,:],
            dims=["time_in_window_idx","window"],
            coords=dict(
                time_in_window_idx=first_window_idx,
                window=output_yrs
            ),
        )
        
        for m,member in tqdm(enumerate(surge_pr.member_id)): #loop over members to compute the dependence
            surge_pr_mem = surge_pr.sel(member_id=member).copy(deep=True).load()
            
            #break ties (where pr=0, ranks are not well defined) #TO_DO!!
            
            #surge_pr_mem['pr'][np.where(surge_pr_mem.pr==0)] = surge_pr_mem['pr'][np.where(surge_pr_mem.pr==0)] + np.random.uniform(0,surge_pr_mem.pr[surge_pr_mem.pr>0].min(dim='time'),(surge_pr_mem.pr==0).sum(dim='time'))
            
            #
            surge_pr_wdws = surge_pr_mem.isel(time=window_idx) #select windows
            
            if season == 'DJF':
                idx = [np.where((surge_pr_wdws.time.isel(window=0).dt.month == month)) for month in [1,2,12]]
                season_idx = np.sort(np.hstack(idx)[0])
                surge_pr_wdws = surge_pr_wdws.isel(time_in_window_idx = season_idx)
            elif season == 'JJA':
                idx = [np.where((surge_pr_wdws.time.isel(window=0).dt.month == month)) for month in [6,7,8]]
                season_idx = np.sort(np.hstack(idx)[0])
                surge_pr_wdws = surge_pr_wdws.isel(time_in_window_idx = season_idx)
            
            
            data_is_complete = np.isfinite(surge_pr_wdws.surge).all(dim='time_in_window_idx') * np.isfinite(surge_pr_wdws.pr).all(dim='time_in_window_idx')
            
            pr_peaks_declustered = declustered_peaks(surge_pr_wdws['pr'],threshold,declus_window_len,dim='time_in_window_idx')
            surge_peaks_declustered = declustered_peaks(surge_pr_wdws['surge'],threshold,declus_window_len,dim='time_in_window_idx')
            
            #compute rank correlations with lag
            ktau_pr_cdon_surge = compute_ktau_in_windows(surge_peaks_declustered,
                                                         rolling_max(surge_pr_wdws['pr'],max_lag*2+1,dim='time_in_window_idx'),
                                                         dim="time_in_window_idx")

            ktau_surge_cdon_pr = compute_ktau_in_windows(pr_peaks_declustered,
                                                         rolling_max(surge_pr_wdws['surge'],max_lag*2+1,dim='time_in_window_idx'),
                                                         dim="time_in_window_idx")

            ktau_both_peaks = compute_ktau_in_windows(surge_peaks_declustered,
                                                         rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx'),
                                                         dim="time_in_window_idx")
            
            #compute UTDC's from daily pairs
            cfg_utdc = compute_utdc_in_windows(surge_pr_wdws['surge'],surge_pr_wdws['pr'],dim="time_in_window_idx",estimator='cfg')
            
            threshold_utdc = compute_utdc_in_windows(surge_pr_wdws['surge'],surge_pr_wdws['pr'],dim="time_in_window_idx",estimator='threshold')
            
            dependence_mem = xr.concat([ktau_pr_cdon_surge,ktau_surge_cdon_pr,ktau_both_peaks],dim='extreme_variate').to_dataset(name='ktau')
            dependence_mem['utdc'] = xr.concat([cfg_utdc,threshold_utdc],dim='estimator')
            dependence_mem['complete_window'] = data_is_complete #store where windows miss data
            
            co_occurring = np.isfinite((rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx')*surge_peaks_declustered))
            
            num_co_occurring_per_month = {}
            for month in np.arange(1,13):
                if month==1:
                    num_co_occurring_pmonth = co_occurring.where(co_occurring.time.dt.month==month).sum(dim='time_in_window_idx')
                else:
                    num_co_occurring_pmonth = xr.concat((num_co_occurring_pmonth,co_occurring.where(co_occurring.time.dt.month==month).sum(dim='time_in_window_idx')),dim='month')
            num_co_occurring_pmonth = num_co_occurring_pmonth.assign_coords({'month':np.arange(1,13)})                             
            
            dependence_mem['num_co_occurring'] = num_co_occurring_pmonth
            #dependence_mem['num_co_occurring'] = np.isfinite((rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx')*surge_peaks_declustered)).sum(dim='time_in_window_idx')
            #^possibly expand this to extreme of one and say 80th percentile of other
            
            dependence_mem['surge_thresholds'] = surge_pr_wdws['surge'].quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds'] = surge_pr_wdws['pr'].quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem = dependence_mem.expand_dims(dim={"member_id": 1}) #add coordinates & dimensions
            dependence_mem = dependence_mem.assign_coords({'extreme_variate':['surge','pr','both'],'statistic':['coef','p'],'estimator':['cfg','.95']})
            
            if m==0: #concatenate results over member_id's
                dependence = dependence_mem
            else:
                dependence = xr.concat((dependence,dependence_mem),dim='member_id')    
            
            dependence.attrs = surge_pr.attrs #keep original attributes and add information on the extremes analysis
            dependence.attrs['window_length'] = str(window_len)
            dependence.attrs['declustering'] = 'Rolling window of '+str(declus_window_len)+' days'
            dependence.attrs['allowed_lag'] = str(max_lag)
            
            #store for all members of this model & experiment
            dependence.to_netcdf(output_fn,mode='w')
            dependence.close()
           

Processing file: CMCC-ESM2_gn_day_ssp245.nc


0it [00:00, ?it/s]

Processing file: CMCC-ESM2_gn_day_ssp585.nc


0it [00:00, ?it/s]

Processing file: CMCC-CM2-SR5_gn_day_ssp245.nc


...

0it [00:00, ?it/s]

/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/

Processing file: EC-Earth3_gr_day_ssp585.nc


0it [00:00, ?it/s]

/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/srv/conda/envs/notebook/lib/python3.10/site-packages/numpy/

In [11]:
np.isfinite((rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx')*surge_peaks_declustered))

<xarray.DataArray (time_in_window_idx: 14600, window: 11, tg: 109)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Coordinates:
    gridcell_lat        (tg) float64 44.3 47.66 40.93 ... 49.91 52.15 43.18
    gridcell_lon        (tg) float64 358.9 357.8 2.25 ... 354.4 355.5 354.4
    time                (time_in_window_idx, window) datetime64[ns] 1861-01-0...
    member_id           <U8 'r1i1p1f1'
  * tg                  (tg) object 'arcachon_eyrac_.csv' ... 'gijon_gijon_sp...
    lat                 (tg) float64 44.66 47.31 41.34 47.29 ... 50.1 51.7 43.56
    lon                 (tg) float64 -1.164 -2.108 2.163 ... -5.014 -5.698
    height              float64 10.0
  * time_in_window_idx  (time_in_window_idx) int64 0 1 2 3 ... 14597 14598 14599
  * window              (window) int64 1880 1900 1920 1940 ... 2040 2060 2080
    quantile            float64 0.99

In [55]:
num_co_occurring_per_month = {}
for month in np.arange(1,13):
    if month==1:
        num_co_occurring_pmonth = co_occ.where(co_occ.time.dt.month==month).sum(dim='time_in_window_idx')
    else:
        num_co_occurring_pmonth = xr.concat((num_co_occurring_pmonth,co_occ.where(co_occ.time.dt.month==month).sum(dim='time_in_window_idx')),dim='month')
num_co_occurring_pmonth = num_co_occurring_pmonth.assign_coords({'month':np.arange(1,13)})                             

In [59]:
num_co_occurring_pmonth.isel(window=-1).sum(dim='month').isel(tg=55)

<xarray.DataArray ()>
array(18.)
Coordinates:
    gridcell_lat  float64 39.81
    gridcell_lon  float64 2.25
    member_id     <U8 'r1i1p1f1'
    tg            <U43 'palma_de_mallorca_palmademallorca_spain.csv'
    lat           float64 39.56
    lon           float64 2.638
    height        float64 10.0
    window        int64 2080
    quantile      float64 0.99

In [ ]:
month_idx = xr.DataArray( #indices of windows
            data=np.isfinite((rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx')*surge_peaks_declustered)).time.dt.month.isel(window=0),
            dims=["time_in_window_idx","window"],
            coords=dict(
                time_in_window_idx=first_window_idx,
                window=output_yrs
            ),
        )

In [9]:
pr_peaks_declustered.isel(tg=0,window=0)[0:30]

<xarray.DataArray 'pr' (time_in_window_idx: 30)>
array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
       0.03083312,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan])
Coordinates:
    gridcell_lat        float64 44.3
    gridcell_lon        float64 358.9
    time                (time_in_window_idx) datetime64[ns] 1861-01-01T12:00:...
    member_id           <U8 'r1i1p1f1'
    tg                  <U19 'arcachon_eyrac_.csv'
    lat                 float64 44.66
    lon                 float64 -1.164
    height              float64 10.0
  * time_in_window_idx  (time_in_window_idx) int64 0 1 2 3 4 ... 25 26 27 28 29
    window              int64 1880
    quantile            float64 0.97
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        includes both liquid and solid phases
    long_name:      Precipitation
    original_name:  PRECIPI
    standard_name:  precipitation_flux
    units:          m